In [ ]:
from flask import Flask, jsonify, redirect, request
from flask_cors import CORS
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from konlpy.tag import Okt
import json

#커스텀 모델 호출을 위한 함수 선언
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)
    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    # return a single tensor value
    return recall
def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림 - 0.5 이하 = 0 / 0.5 초과 = 1
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정
    # True Positive = 실제 값과 예측 값이 모두 1(Positive)인 경우
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)
    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon() = 'divide by zero error' 예방차원에서 작은 수를 더함
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    # return a single tensor value
    return precision
def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon() = divide by zero error' 예방차원에서 작은 수를 더함
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    # return a single tensor value
    return _f1score

# Server
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

# Ai Function 
@app.route("/use_model", methods=['POST'])
def use_model():
	# Chrome Extension으로부터 전송받은 데이터
	examples_temp = request.get_json()
	examples = examples_temp["text"]
	okt = Okt()
	ex_morpheme = []
	# 문장 형태소 분석
	for text in examples:
		mor_data = ""
		for word_tag in okt.pos(text, norm=True, stem=True):
			if word_tag[1] in ['Noun', 'Verb', 'VerbPrefix', 'Adjective', 'Determiner', 'Adverb', 'Exclamation', 'KoreanParticle']:
				mor_data += word_tag[0]
				mor_data += " "
		ex_morpheme.append(mor_data)
	# 문장 및 사용될 단어의 최대 길이/갯수 설정
	maxlen = 50
	max_words = 10000
	# 생성된 형태소 배열 토큰화 및 정수 인코딩
	tokenizer = Tokenizer(num_words=max_words)
	with open('wordIndex.json') as json_file:
		word_index = json.load(json_file)
		tokenizer.word_index = word_index
	tokenizer.fit_on_texts(word_index) # 빈도수 기준으로 단어 집합 생성
	sequences = tokenizer.texts_to_sequences(ex_morpheme) # 단어 빈도수가 높은 순으로 번호 부여
	# 문장 별 길이를 동일하게 맞추기 위한 패딩
	x_test = pad_sequences(sequences, maxlen=maxlen)
	# 모델을 불러와 데이터로부터 욕설 가능성 계산 후 반환
	model = load_model('model.h5', custom_objects = {"precision": precision, "recall" : recall, "f1score" : f1score})
	prob = model.predict(x_test)
	for i in range(0, len(x_test)):
		print(examples[i], ":", round(prob[i][0] * 100, 1), "%의 확률로 욕설입니다.")            
	prob_json = { "prob": []}
	for i in range(0, len(x_test)):
		prob_json["prob"].append(float(prob[i][0]))
	return prob_json

# Server address
if __name__ == "__main__":
	app.run(host="127.0.0.1", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Dec/2020 16:31:44] "OPTIONS /use_model HTTP/1.1" 200 -


ㅋㅋ : 21.9 %의 확률로 욕설입니다.
ㅋㅋ : 21.9 %의 확률로 욕설입니다.
ㅋㅋㅋ - dc App : 24.5 %의 확률로 욕설입니다.
이스미 추! - dc App : 49.6 %의 확률로 욕설입니다.
ㅋㅋㅋ - dc App : 24.5 %의 확률로 욕설입니다.
개웃김ㅋㅋㅋ - dc App : 45.7 %의 확률로 욕설입니다.
빵! 터졌엌 - dc App : 19.2 %의 확률로 욕설입니다.
좆노잼 눈물쥐촉촉단 : 95.8 %의 확률로 욕설입니다.
페이커대박ㅋㅋ - dc App : 16.6 %의 확률로 욕설입니다.
탑캐리급 노잼 : 26.2 %의 확률로 욕설입니다.
ㅋㅋㅋㅋ - dc App : 24.5 %의 확률로 욕설입니다.
애썼는데 재미는 없노.. - dc App : 17.8 %의 확률로 욕설입니다.
ㅋㅋㅋㅋㅋㅋㅋㅋ : 24.5 %의 확률로 욕설입니다.
살면서 이렇게 웃어본적 처음이다 크크크 - dc App : 23.0 %의 확률로 욕설입니다.
담원아니라 ㅁㅈㅎ : 61.7 %의 확률로 욕설입니다.
ㅋㅋㅋㅋㅋ - dc App : 24.5 %의 확률로 욕설입니다.
진짜 시발 개찐따같아 인벤으로꺼져 : 99.9 %의 확률로 욕설입니다.
ㅈㄴ기네 슼갈씨발련 : 85.4 %의 확률로 욕설입니다.
개웃ㅋㅋㅋ - dc App : 24.5 %의 확률로 욕설입니다.
앜ㅋㅋ - dc App : 59.3 %의 확률로 욕설입니다.
앜ㅋㅋㅋㅋㅋㅋ : 66.0 %의 확률로 욕설입니다.
그림일기) 싸맥의 역습.jpg : 27.4 %의 확률로 욕설입니다.
2020년 올해의 이슈는? : 30.0 %의 확률로 욕설입니다.
리그 오브 레전드 관련 내용이 있어야 합니다. [148] : 14.3 %의 확률로 욕설입니다.
그림일기) 싸맥의 역습.jpg [21] : 27.4 %의 확률로 욕설입니다.
앰비션의 씨맥 사건 소신 발언 [107] : 17.0 %의 확률로 욕설입니다.
이시각 제일 웃긴새끼.jpg [126] : 99.2 %의 확률로 욕설입니다.
대깨맥 논리대로면 조규남도 영구제

 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
5개월뒤 : 19.

 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
그 약속 꼭 지켜라. : 17.2 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.

127.0.0.1 - - [15/Dec/2020 16:31:56] "POST /use_model HTTP/1.1" 200 -


%의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
LCK 써머  : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 확률로 욕설입니다.
 : 19.7 %의 